In [13]:
import pandas as pd
import json
from tempfile import NamedTemporaryFile
from pathlib import Path
from utils.matching import Matcher, load_segments
df = pd.read_csv("006_crawling_juznevesti.csv")
segs = pd.read_csv("011_segments.csv").rename(columns={"predicted":"text"})
segs["asr_transcription"] = ""
segs["ratio"] = ""
segs["webpage_transcript"] = ""
for path in segs.file.unique():
    try:
        cond = segs.file == path
        original_transcription = " ".join(df[df.path==path].transcript)

        with NamedTemporaryFile() as f:
            asr_file = Path(f.name)
            asr_results = segs.loc[cond, :].to_json(asr_file, orient="records", lines=True)
            asr_results = load_segments(asr_file)
        with NamedTemporaryFile() as g:
            transcription_file = Path(g.name)
            transcription_file.write_text(original_transcription)
            matcher = Matcher(transcription_file)
        positions = matcher.match(asr_results)
        segs.loc[cond, "asr_transcription"] = [" ".join(i.segment.text) for i in positions]
        segs.loc[cond, "webpage_transcript"] = [" ".join(
            original_transcription.split()[i.corp_start:i.corp_end+1]
        ) for i in positions]
        segs.loc[cond, "ratio"] = [i.ratio for i in positions]
    except AttributeError:
        continue
segs.to_json("013_segments_matched.jsonl", orient="records", lines=True)

Skipping 0'th segment because too little words: 6
Skipping 0'th segment because too little words: 7
ERROR: no candidates found!
ERROR: no candidates found!
ERROR: no candidates found!
ERROR: no candidates found!


In [12]:
import pandas as pd
df = pd.read_json("013_segments_matched.jsonl", orient="records", lines=True)
df.file.unique().shape

(293,)

In [8]:
original_transcription.split()[19:55]

['Od',
 'kada',
 'je',
 '2004.',
 'usvojen',
 'Zakon',
 'o',
 'slobodnom',
 'pristupu',
 'informacijama',
 'od',
 'javnog',
 'značaja',
 'postao',
 'je',
 'nezaobilazan',
 'alat',
 'za',
 'mnoge',
 'novinare',
 'ali',
 'i',
 'građanske',
 'aktiviste.',
 'Poput',
 'šale,',
 'kada',
 'dečak',
 'pita',
 'roditelje',
 'kako',
 'ste',
 'vi',
 'slali',
 'emailove',
 'pre']

In [15]:
positions[0]

Position(corp_start=19, corp_end=55, seg_start=0, seg_end=37, ratio=0.8493150684931506, segment=Segment(file='audio/fTfyE4ZdZqY.wav', start=16.3, end=33.1, text=['od', 'kada', 'je', '2004.', 'usvojen', 'zakon', 'o', 'slobodnom', 'pristupu', 'informacijama', 'od', 'javnog', 'značaja', 'postao', 'je', 'nezaobila', 'za', 'nalad', 'za', 'mnoge', 'novinare', 'ali', 'i', 'građanske', 'aktiviste', 'e', 'sada', 'poput', 'šale', 'kada', 'dečak', 'pita', 'roditelje', 'kako', 'ste', 'vi', 'slali']))